In [ ]:
#-*- coding: utf-8 -*-
#import sys 
#reload(sys)
#sys.setdefaultencoding('utf-8')

import requests
from pprint import pprint


In [9]:
# kakao api
APP_KEY = "${API_KEY}"
kakao_url = "https://dapi.kakao.com/v2/local/search/address.json"
kakao_headers = {"Authorization" : "KakaoAK {}".format(APP_KEY)}

# naver api
client_id = "Q9qo7SjCK9jHPd7riqzE"
client_secret = "QOBbQMJe1N"
naver_url = "https://openapi.naver.com/v1/map/geocode" 
naver_headers = {"X-Naver-Client-Id" : "{}".format(client_id) , "X-Naver-Client-Secret": "{}".format(client_secret)}

In [10]:
def kakao_req(params, method):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % kakao_url)
#     print('Headers: %s' % kakao_headers)
#     print('QueryString: %s' % params)

    if method == 'GET':
        return requests.get(kakao_url, headers=kakao_headers, params=params)
    
def naver_req(params, method):
#     print('HTTP Method: %s' % method)
#     print('Request URL: %s' % naver_url)
#     print('Headers: %s' % naver_headers)
#     print('QueryString: %s' % params)
    
    if method == 'GET':
        return requests.get(naver_url, headers=naver_headers, params=params)

In [11]:
def get_location(address):

    params = {"query": "{}".format(address)}

    response = naver_req(params, 'GET')
        

#    #kakao parsing
#     pprint(data['meta']['total_count'])
#     pprint(data["documents"][0]["x"])
#     pprint(data["documents"][0]["y"])
    
        
    return response

In [12]:
import findspark
findspark.init()
import pyspark
import os
import numpy as np

In [13]:
#sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [15]:
basic_folder = '/home/project/real_estate/data/apartment_rent_data/'

file_list = os.listdir(basic_folder)
path_list=[]
for file in file_list:
    print(file)
    path_list.append(basic_folder + file)
    
print(path_list)

apartment_rent_201207.csv
apartment_rent_201504.csv
apartment_rent_201309.csv
apartment_rent_201502.csv
apartment_rent_201805.csv
apartment_rent_201401.csv
apartment_rent_201102.csv
apartment_rent_201410.csv
apartment_rent_201105.csv
apartment_rent_201301.csv
apartment_rent_201206.csv
apartment_rent_201101.csv
apartment_rent_201611.csv
apartment_rent_201302.csv
apartment_rent_201306.csv
apartment_rent_201403.csv
apartment_rent_201209.csv
apartment_rent_201109.csv
apartment_rent_201703.csv
apartment_rent_201608.csv
apartment_rent_201806.csv
apartment_rent_201205.csv
apartment_rent_201112.csv
apartment_rent_201208.csv
apartment_rent_201512.csv
apartment_rent_201506.csv
apartment_rent_201204.csv
apartment_rent_201803.csv
apartment_rent_201510.csv
apartment_rent_201606.csv
apartment_rent_201708.csv
apartment_rent_201201.csv
apartment_rent_201503.csv
apartment_rent_201505.csv
apartment_rent_201509.csv
apartment_rent_201710.csv
apartment_rent_201409.csv
apartment_rent_201111.csv
apartment_re

In [16]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("header", "true").option("inferSchema", "true").load(path_list)

In [17]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- build_year: integer (nullable = true)
 |-- deposit: string (nullable = true)
 |-- rental_fee: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: string (nullable = true)
 |-- code: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- sub_city: string (nullable = true)
 |-- dong: string (nullable = true)
 |-- name: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- exclusive_private_area: double (nullable = true)
 |-- address: string (nullable = true)
 |-- detailed_address: string (nullable = true)



In [18]:
df.show()
df.count()

+---+----------+----------+----------+----+-----+-----+-----+-----+--------+------+---------------+-----+----------------------+----------------+----------------+
|_c0|build_year|   deposit|rental_fee|year|month|  day| code| city|sub_city|  dong|           name|floor|exclusive_private_area|         address|detailed_address|
+---+----------+----------+----------+----+-----+-----+-----+-----+--------+------+---------------+-----+----------------------+----------------+----------------+
|  0|      2008|    35,000|       120|2015|    3| 1~10|11110|서울특별시|     종로구|   사직동|광화문풍림스페이스본(9-0)|    2|                 94.51|  서울특별시 종로구  사직동|               9|
|  1|      2008|    72,000|        50|2015|    3| 1~10|11110|서울특별시|     종로구|   사직동|광화문풍림스페이스본(9-1)|    2|                163.33|  서울특별시 종로구  사직동|             9-1|
|  2|      2008|    60,000|       100|2015|    3| 1~10|11110|서울특별시|     종로구|   사직동|광화문풍림스페이스본(9-1)|    2|                 150.4|  서울특별시 종로구  사직동|             9-1|
|  3|      2008|    30

4631930

In [20]:
#filter_df = df.filter((df["detailed_address"] == "") | df["detailed_address"].isNull() | isnan(df["detailed_address"]))
#filter_df.show()
#filter_df.count() == 36

filter_df = df.na.drop()



In [21]:
from pyspark.sql.functions import *

In [22]:
unique_date_df = filter_df.groupBy(filter_df.year,filter_df.month, filter_df.day).agg(count("day").alias('count'))

unique_date_list = unique_date_df.collect()

new_date_list = []

for unique_date in unique_date_list:
    if unique_date[1] < 10 :
        if int(unique_date[2].split('~')[0]) < 10 : 
            date = str(unique_date[0]) + '-' + '0' + str(unique_date[1]) + '-' + '0' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
        else:
            date = str(unique_date[0]) + '-' + '0' + str(unique_date[1]) + '-' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
        
    else:
        if int(unique_date[2].split('~')[0]) < 10 : 
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' + '0' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
    
        else:
            date = str(unique_date[0]) + '-' + str(unique_date[1]) + '-' +unique_date[2].split('~')[0] + 'T' + "00:00:00"
    
    
    new_date_list.append([unique_date[0], unique_date[1],unique_date[2],date])
    
    
unique_date_df = sc.parallelize(new_date_list).toDF(['year','month','day','date'])

In [23]:
added_date_df = filter_df.join(unique_date_df, ['year','month','day']).sort('date')
added_date_df.show()
added_date_df.count()

+----+-----+----+---+----------+----------+----------+-----+-----+--------+-----+---------------+-----+----------------------+---------------+----------------+-------------------+
|year|month| day|_c0|build_year|   deposit|rental_fee| code| city|sub_city| dong|           name|floor|exclusive_private_area|        address|detailed_address|               date|
+----+-----+----+---+----------+----------+----------+-----+-----+--------+-----+---------------+-----+----------------------+---------------+----------------+-------------------+
|2011|    1|1~10| 67|      2004|    35,000|         0|11110|서울특별시|     종로구|  숭인동|        롯데캐슬천지인|   23|                111.73| 서울특별시 종로구  숭인동|              76|2011-01-01T00:00:00|
|2011|    1|1~10| 41|      1993|    13,000|         0|11110|서울특별시|     종로구|  창신동|          창신쌍용2|    1|                  54.7| 서울특별시 종로구  창신동|             703|2011-01-01T00:00:00|
|2011|    1|1~10| 48|      1968|       500|        50|11110|서울특별시|     종로구|  창신동|            동대문|   

4630954

In [24]:
new_df = added_date_df.groupBy(added_date_df.address,added_date_df.detailed_address).agg(count("detailed_address").alias("count"))
new_df.show()
new_df.count()

+-----------------+----------------+-----+
|          address|detailed_address|count|
+-----------------+----------------+-----+
|   서울특별시 종로구  숭인동|             766|  273|
|  서울특별시 동대문구  제기동|             153|  388|
|  서울특별시 동대문구  장안동|           329-2| 1002|
|   서울특별시 성북구  종암동|           104-5|   39|
|   서울특별시 송파구  풍납동|             511|  140|
|    부산광역시 북구  화명동|            2264|  811|
|   부산광역시 사상구  학장동|             819|   87|
|   대구광역시 수성구  황금동|             240|  155|
|    인천광역시 남구  관교동|            13-9|  175|
|   인천광역시 남동구  남촌동|             651|   17|
|   인천광역시 남동구  논현동|           755-4|  800|
|   인천광역시 부평구  삼산동|           453-1|  875|
|   인천광역시 부평구  부개동|           477-2|  291|
|   인천광역시 계양구  병방동|             383|  153|
|    광주광역시 남구  주월동|           543-9|  306|
|    광주광역시 북구  운암동|              68|   98|
|   대전광역시 유성구  지족동|             834|   58|
|울산광역시 울주군 온양읍 운화리|          163-22|   60|
|    경기도 의정부시  민락동|           692-1|  411|
|      경기도 부천시  상동|           571-1|  200|
+----------

38250

In [25]:
sum = new_df.select(sum("count"))

In [26]:
sum.show()

+----------+
|sum(count)|
+----------+
|   4630954|
+----------+



In [27]:
unique_address_list = new_df.collect()
#print unique_address_list[0][0]

KeyboardInterrupt: 

In [19]:
location_list = []

# checking why no result data
# response = get_location("경상북도 영천시 신령면 완전리") -> 신녕면
# print response.status_code
# data = response.json()
# print data

index = 1.0;
length = len(unique_address_list)
count = 0;

print length 

for address in unique_address_list:    
    full_address = (address[0] + " " + address[1]).encode('utf8')
    #print str("full " + full_address)
    response = get_location(full_address)

    if response.status_code == 200 :
        
        data = response.json()
        
        
        x = data['result']['items'][0]['point']['x']
        y = data['result']['items'][0]['point']['y']
    
        location_list.append([address[0],address[1],y,x])
        
    else: 
        print str("error address: " + full_address)
        count += 1
        print count
        
        location_list.append([address[0],address[1] ,np.nan,np.nan])
    
    if(index % 100 == 0):
        print ("%0.2f" % (index / length * 100.0)) + "% completed"
        
    index += 1.0

     

9480
1.05% completed
2.11% completed
3.16% completed
4.22% completed
5.27% completed
6.33% completed
7.38% completed
8.44% completed
9.49% completed
error address: 부산광역시 금정구  장전동 74-93
1
10.55% completed
11.60% completed
error address: 서울특별시 관악구  봉천동 863-6
2
12.66% completed
13.71% completed
14.77% completed
15.82% completed
16.88% completed
17.93% completed
18.99% completed
20.04% completed
21.10% completed
22.15% completed
23.21% completed
24.26% completed
25.32% completed
26.37% completed
27.43% completed
28.48% completed
29.54% completed
30.59% completed
31.65% completed
32.70% completed
33.76% completed
34.81% completed
35.86% completed
36.92% completed
37.97% completed
39.03% completed
40.08% completed
41.14% completed
42.19% completed
43.25% completed
44.30% completed
45.36% completed
46.41% completed
47.47% completed
48.52% completed
49.58% completed
50.63% completed
51.69% completed
52.74% completed
53.80% completed
54.85% completed
55.91% completed
56.96% completed
58.02% com

In [20]:
#print location_list[0]


#checking_df = df.filter(df.address == "인천광역시 서구  왕길동").filter(df.detailed_address == "가-81-1")

#checking_df.show()
#print(checking_df.count())

                
location = sc.parallelize(location_list).toDF(['address','detailed_address','latitude','longitude'])
location.show()

+-----------------+----------------+----------+-----------+
|          address|detailed_address|  latitude|  longitude|
+-----------------+----------------+----------+-----------+
|   대전광역시 유성구  지족동|           863-2|36.3825628|127.3205555|
|  경기도 평택시 안중읍 안중리|          252-33|36.9881887|126.9255043|
|  경기도 화성시 남양읍 남양리|          1265-3|37.2101135|126.8181706|
|   부산광역시 금정구  장전동|          364-24|35.2349315|129.0886999|
|   부산광역시 금정구  장전동|          227-12|35.2372391| 129.087994|
|  경기도 안산시 상록구  사동|          1197-4|37.3017055|126.8511012|
|     경기도 이천시  중리동|           211-8|37.2784762| 127.444716|
|경상남도 창원시 성산구  상남동|            11-6| 35.222296|128.6851114|
|    인천광역시 남구  도화동|            90-2|37.4677795|126.6593777|
|  서울특별시 동대문구  이문동|          325-20|37.5944677|127.0593227|
|부산광역시 기장군 기장읍 대라리|           500-6|35.2424389|129.2117922|
|  서울특별시 서대문구  창천동|           29-81| 37.556057|126.9382413|
|   서울특별시 강북구  수유동|            48-1|37.6344091|127.0228738|
|   서울특별시 강동구  암사동|           502-3|37.5

In [21]:
location.count()

9480

In [22]:
# error 
#new_df_id = new_df.withColumn('id', monotonically_increasing_id())
#new_df_id.show()
#new_df_id.count()
#new_df_id.filter(new_df_id.id == '10000').show()
#add_df.count()


#location_id = location.withColumn('id', row_number().over (Window.partitionBy().partitionBy(location['logitude']).orderBy(lit(1))))
#location_id.show()
#location_id.count()

#location_id.filter(location_id.id == '37777').show()


#add_df = new_df_id.select('address', 'detailed_address','id').join(location_id , new_df_id.id == location_id.id).drop(new_df_id.id)
#add_df.orderBy('id', ascending=True).show()
#add_df.count()

In [23]:
joined_df = added_date_df.join(location.select('address','detailed_address','latitude','longitude'), ['address','detailed_address'])

In [24]:
joined_df.show()
joined_df.count()

+----------------+----------------+----+-----+-----+---+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|_c0|build_year|deposit|rental_fee|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+---+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|경기도 수원시 영통구  이의동|            1330|2012|   11| 1~10|  1|      2012|    500|        55| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|    7|                 28.96|2012-11-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|2013|    1| 1~10|  1|      2012|  7,000|         0| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|   12|                 28.96|2013-01-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이

380095

In [25]:
drop_index_df = joined_df.drop("_c0")
drop_index_df.show()
drop_index_df.count()

+----------------+----------------+----+-----+-----+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|build_year|deposit|rental_fee|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|경기도 수원시 영통구  이의동|            1330|2012|   11| 1~10|      2012|    500|        55| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|    7|                 28.96|2012-11-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|2013|    1| 1~10|      2012|  7,000|         0| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|   12|                 28.96|2013-01-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|

380095

In [26]:
final_apartment_df = drop_index_df.na.drop()
final_apartment_df = final_apartment_df.withColumn('deposit',regexp_replace('deposit',"\\,", "")).withColumn('rental_fee',regexp_replace('rental_fee',"\\,", ""))
final_apartment_df.show()
final_apartment_df.count()

+----------------+----------------+----+-----+-----+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|         address|detailed_address|year|month|  day|build_year|deposit|rental_fee|city|sub_city|dong| code|        name|floor|exclusive_private_area|               date|  latitude|  longitude|
+----------------+----------------+----+-----+-----+----------+-------+----------+----+--------+----+-----+------------+-----+----------------------+-------------------+----------+-----------+
|경기도 수원시 영통구  이의동|            1330|2012|   11| 1~10|      2012|    500|        55| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|    7|                 28.96|2012-11-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|2013|    1| 1~10|      2012|   7000|         0| 경기도| 수원시 영통구| 이의동|41117|광교서희스타힐스오피스텔|   12|                 28.96|2013-01-01T00:00:00|37.2924897|127.0491615|
|경기도 수원시 영통구  이의동|            1330|

379966

In [28]:
final_apartment_df.write.csv("final_apartment_rent")